#  RECOMMENDATION SYSTEM   (collabrative filtering)

In [1]:
import numpy as np
import pandas as pd
from numpy import array
import math

In [2]:
movies=pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies['genres']=[x.split('|') for x in movies['genres']]
movies['title']=[x.split('(')[0].replace('(\(\d\d\d\d\))', '').strip() for x in movies['title']]
movies.head()

,movieId,title,genres
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men,"[Comedy, Romance]"
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II,[Comedy]


In [4]:
s=set()
for x in zip(movies['genres']):
    for i in x[0]:
        s.add(i)

print(list(s))    

['Action', 'Musical', 'Documentary', 'Children', 'Fantasy', 'Adventure', 'Animation', 'Sci-Fi', 'War', 'Film-Noir', 'Comedy', 'Mystery', 'Horror', 'Crime', '(no genres listed)', 'Drama', 'Thriller', 'IMAX', 'Western', 'Romance']


In [5]:
# for index, row in movies.iterrows():
#     for genre in row['genres']:
#         movies.at[index, genre] = 1
        
# movies = movies.fillna(0)
# movies.head()

In [6]:
ratings=pd.read_csv('ratings.csv')
ratings= ratings.drop('timestamp', 1)
ratings.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [7]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,rating,title
0,5.0,"Breakfast Club, The"
1,3.5,Toy Story
2,2.0,Jumanji
3,5.0,Pulp Fiction
4,4.5,Akira


In [8]:
inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,movieId,title,genres,rating
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",3.5
1,2,Jumanji,"[Adventure, Children, Fantasy]",2.0
2,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",5.0
3,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",4.5
4,1968,"Breakfast Club, The","[Comedy, Drama]",5.0


In [9]:
users = ratings[ratings['movieId'].isin(inputMovies['movieId'].tolist())]
users.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


In [10]:
k=users.groupby('userId')
uqusers=list(set(users['userId'].tolist()))

In [11]:
mov5us=[x  for x in uqusers if len(k.get_group(x))==5]
mov5us

[75,
 106,
 686,
 815,
 1040,
 1130,
 1502,
 1599,
 1625,
 1950,
 2065,
 2128,
 2432,
 2791,
 2839,
 2948,
 3025,
 3040,
 3186,
 3271,
 3429,
 3734,
 4099,
 4208,
 4282,
 4292,
 4415,
 4586,
 4725,
 4818,
 5104,
 5165,
 5547,
 6082,
 6207,
 6366,
 6482,
 6530,
 7235,
 7403,
 7641,
 7996,
 8008,
 8086,
 8245,
 8572,
 8675,
 9101,
 9358,
 9663,
 9994,
 10248,
 10315,
 10368,
 10607,
 10707,
 10863,
 11314,
 11399,
 11769,
 11827,
 12069,
 12120,
 12211,
 12325,
 12916,
 12921,
 13053,
 13142,
 13260,
 13366,
 13768,
 13888,
 13923,
 13934,
 14529,
 14551,
 14588,
 14984,
 15137,
 15157,
 15466,
 15670,
 15834,
 16292,
 16456,
 16506,
 17246,
 17438,
 17501,
 17502,
 17666,
 17735,
 17742,
 17757,
 17854,
 17897,
 17944,
 18301,
 18509,
 18765,
 18905,
 19208,
 19381,
 19607,
 19890,
 20278,
 20319,
 20341,
 20444,
 20650,
 20708,
 20759,
 20838,
 20969,
 21023,
 21148,
 21242,
 21362,
 21473,
 21475,
 21819,
 22043,
 22180,
 22883,
 22950,
 22952,
 23297,
 23332,
 23437,
 23467,
 23534,


In [12]:
columns=[]
columns.append('UserId')
columns.extend(inputMovies['movieId'].tolist())

data=[]

for i in mov5us:
    l=k.get_group(i)
    j=l['userId'].tolist()[0]
    col=l['rating'].tolist()
    li=[]
    li.append(j)
    li.extend(col)
    data.append(li)
    
subsetuser=pd.DataFrame(columns=columns,data=data)
subsetuser.head()

,UserId,1,2,296,1274,1968
0,75,5.0,3.5,5.0,4.5,5.0
1,106,2.5,3.0,3.5,3.0,3.5
2,686,4.0,3.0,4.0,4.0,5.0
3,815,4.5,3.0,5.0,3.0,4.5
4,1040,3.0,1.5,3.5,3.0,4.0


<img src="capture.png">

In [13]:
userrateing=array(inputMovies['rating'].tolist())
x=[userrateing.mean()]*len(userrateing)

pearson_coff=[]
for index, row in subsetuser.iterrows():
    tuserrat=array(row.tolist()[1:])
    y=[tuserrat.mean()]*len(userrateing)
    
   
    sxy=((userrateing-x)*(tuserrat-y)).sum()
    sxx=math.sqrt((pow(userrateing-x,2)).sum())
    syy=math.sqrt((pow(tuserrat-y,2)).sum())

    if sxx!=0 or sxy!=0:
        pearson_coff.append(sxy/(sxx*syy))
    else:
        pearson_coff.append(0)
    
        
subsetuser['pp_coff']=pearson_coff
subsetuser.head()

C:\Users\vivek\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


,UserId,1,2,296,1274,1968,pp_coff
0,75,5.0,3.5,5.0,4.5,5.0,0.827278
1,106,2.5,3.0,3.5,3.0,3.5,0.586009
2,686,4.0,3.0,4.0,4.0,5.0,0.832050
3,815,4.5,3.0,5.0,3.0,4.5,0.576557
4,1040,3.0,1.5,3.5,3.0,4.0,0.943456


In [14]:
similaruserlist=subsetuser.sort_values('pp_coff',ascending=False)[0:50]
similaruserlist

,UserId,1,2,296,1274,1968,pp_coff
568,96562,3.0,2.0,4.0,4.0,4.0,0.986690
1329,232983,4.0,3.0,5.0,5.0,5.0,0.986690
903,162014,3.5,3.0,4.0,4.0,4.0,0.986690
199,36711,4.0,3.0,5.0,5.0,5.0,0.986690
952,170007,4.5,4.0,5.0,5.0,5.0,0.986690
749,132358,3.5,3.0,4.0,4.0,4.0,0.986690
685,120010,3.0,2.0,4.0,4.0,4.0,0.986690
110,20650,3.0,2.5,3.5,3.5,3.5,0.986690
663,115000,3.0,1.0,5.0,5.0,5.0,0.986690
258,45640,4.5,4.0,5.0,5.0,5.0,0.986690


In [15]:
topUsersRating=similaruserlist.merge(ratings, left_on='UserId', right_on='userId', how='inner')
topUsersRating.head()

,UserId,1,2,296,1274,1968,pp_coff,userId,movieId,rating
0,96562,3.0,2.0,4.0,4.0,4.0,0.98669,96562,1,3.0
1,96562,3.0,2.0,4.0,4.0,4.0,0.98669,96562,2,2.0
2,96562,3.0,2.0,4.0,4.0,4.0,0.98669,96562,18,3.0
3,96562,3.0,2.0,4.0,4.0,4.0,0.98669,96562,21,4.0
4,96562,3.0,2.0,4.0,4.0,4.0,0.98669,96562,24,2.0


In [16]:
topUsersRating['weightrating']=topUsersRating['pp_coff']*topUsersRating['rating']
topUsersRating=topUsersRating.drop(columns=l['movieId'].tolist())
topUsersRating=topUsersRating.drop(columns=['userId','UserId'])
topUsersRating.head()

,pp_coff,movieId,rating,weightrating
0,0.98669,1,3.0,2.960071
1,0.98669,2,2.0,1.973381
2,0.98669,18,3.0,2.960071
3,0.98669,21,4.0,3.946761
4,0.98669,24,2.0,1.973381


In [17]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['pp_coff','weightrating']]
tempTopUsersRating.columns = ['sumpp_coff','sum_weightedRating']
tempTopUsersRating.head()

,sumpp_coff,sum_weightedRating
movieId,,
1,48.629129,174.108534
2,48.629129,114.421897
3,15.581322,41.428504
4,1.932512,3.865024
5,9.710614,19.519327


In [18]:
tempTopUsersRating['recomm_score']=tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sumpp_coff']
tempTopUsersRating = tempTopUsersRating.sort_values(by='recomm_score', ascending=False)
tempTopUsersRating.head(10)

,sumpp_coff,sum_weightedRating,recomm_score
movieId,,,
32892,1.956554,9.782769,5.0
80928,0.970725,4.853627,5.0
5028,0.969864,4.849318,5.0
43,0.960769,4.803845,5.0
8684,0.969864,4.849318,5.0
3749,0.980581,4.902903,5.0
8601,0.969864,4.849318,5.0
42217,0.969864,4.849318,5.0
8484,0.969864,4.849318,5.0


<h3>These top 10 movies are recommended more can be also put into list from tempTopUserRating</h3>

# Other ways of calculation recommendation score

<ul>
    <li>pearson's coeffient</li>
    <li>Cosine similarity</li>
    <li>Euclidian distance</li>
    <li></li>
</ul>

<img src="capture2.png">
<img src="capture3.png">